In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
import json
from unsloth.chat_templates import get_chat_template, train_on_responses_only
from pathlib import Path
from transformers.utils import can_return_loss, infer_framework
import inspect
import datasets
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
from peft import AutoPeftModelForCausalLM

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained("./lora_model_ign2o4dn", device_map=0, torch_dtype=torch.bfloat16)
model.eval()
tokenizer = AutoTokenizer.from_pretrained("./lora_model_ign2o4dn", fast=True)
print(tokenizer.chat_template)

In [ ]:
tokenizer.push_to_hub("fancyfeast/add_source_lora_model", private=True)
model.push_to_hub("fancyfeast/add_source_lora_model", private=True)

In [3]:
# # Add the PROMPT to the system message, so it's baked into the model making it easier to use and harder to give the wrong prompt (if it changes between versions)
# tokenizer = AutoTokenizer.from_pretrained("./lora_model", fast=True)
# assert isinstance(tokenizer.chat_template, str)
# tokenizer.chat_template = tokenizer.chat_template.replace("{{- system_message }}", '{{- ' + json.dumps(PROMPT) + ' }}')

# convo = [
# 	{"role": "user", "content": "A photograph"},
# ]

# foo = tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = True)
# assert isinstance(foo, str)
# print(foo)
# input_ids2 = tokenizer.encode(foo, return_tensors = "pt", truncation=False, max_length = 1024, add_special_tokens=False).cuda()
# print(input_ids2)

In [ ]:
# Merge the LORA
model = model.merge_and_unload(progressbar=True)

In [ ]:
# Push to hub and save locally
#model.save_pretrained("big-asp-caption-watermark-rephraser-hf")
#tokenizer.save_pretrained("big-asp-caption-watermark-rephraser-hf")

model.push_to_hub("fancyfeast/big-asp-caption-add-source-rephraser", private=True)
tokenizer.push_to_hub("fancyfeast/big-asp-caption-add-source-rephraser", private=True)


In [ ]:
# Convert to gguf
# $CONDA_PREFIX/lib/python3.11/site-packages/bin/convert_hf_to_gguf.py --outfile big-asp-captioner-rephraser.gguf --outtype bf16 --model-name big-asp-caption-rephraser big-asp-caption-rephraser-hf
# Had to use docker, the script was not working on my machine
# docker run -v (pwd)/recaptioning:/foo --rm -it ghcr.io/ggerganov/llama.cpp:full --convert --outfile /foo/big-asp-caption-rephraser.gguf --outtype bf16 --model-name big-asp-caption-rephraser /foo/big-asp-caption-rephraser-hf

In [ ]:
chat_tokenizer = get_chat_template(
	tokenizer,
	chat_template = "llama-3.1",
)

In [ ]:
PROMPT = """Please edit the user's provided image descriptions following the guidelines below:
1. The edits should be minimal and not affect the details or accuracy of the description.
2. Remove any mention of the image's resolution, but don't remove information about the image's quality.
3. Edit out any self-referential language. For example: "this is a digital painting" -> "a digital painting", "In this photo a woman stands" -> "photo of a woman standing", etc.
4. Randomly swap in informal synonyms for things like "penis", "vulva", etc.
5. Do not modify anything in quotes that are describing text in the image.
6. Randomly swap the word "photograph" to "photo".
7. Remove any duplicates from the description if the description repeats itself.
8. When you make edits, make sure to maintain the original meaning of the sentence, and minimize the number of changes.
9. Only update the grammer if necessary. Do NOT fix any grammar mistakes or oddness that were in the original description. Some of them may be MidJourney prompts or lists of tags.

Respond with only the edited image description.
"""

CAPTION = ""

convo = [
	{"role": "system", "content": PROMPT},
	{"role": "user", "content": CAPTION},
]

foo = tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = True)
assert isinstance(foo, str)
print(foo)
input_ids = tokenizer.encode(foo, return_tensors = "pt", truncation=False, max_length = 1024, add_special_tokens=False).cuda()
print(input_ids)

In [ ]:
with torch.no_grad():
	generate_ids = model.generate(input_ids, max_new_tokens=512, suppress_tokens=None)
	generate_ids = generate_ids[:, input_ids.shape[1]:]
	if generate_ids[0][-1] == tokenizer.eos_token_id or generate_ids[0][-1] == tokenizer.convert_tokens_to_ids("<|eot_id|>"):
		generate_ids = generate_ids[:, :-1]
	
	print(f"Generated {generate_ids.shape[1]} tokens")

	caption = tokenizer.batch_decode(generate_ids, skip_special_tokens=False, clean_up_tokenization_spaces=False)[0]
	print(caption)